In [ ]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns

In [ ]:
# Import the data 
df = pd.read_csv('Stanley Cup Data - project_dataset.csv')

# Sorting the data by year 
df.sort_values(by = "Season Start")


# Replacing the stanely cup column with a one if that team one the championship
# List of stanely cup winners: https://www.nhl.com/news/nhl-stanley-cup-champions-winners-complete-list-287705398

# Masking the dataset for each season
Season_1998 = df[df['Season Start'] == 1998]
Season_1999 = df[df['Season Start'] == 1999]
Season_2000 = df[df['Season Start'] == 2000]
Season_2001 = df[df['Season Start'] == 2001]
Season_2002 = df[df['Season Start'] == 2002]
Season_2003 = df[df['Season Start'] == 2003]
# Note the 2004 season was canceled
Season_2005 = df[df['Season Start'] == 2005]
Season_2006 = df[df['Season Start'] == 2006]
Season_2007 = df[df['Season Start'] == 2007]
Season_2008 = df[df['Season Start'] == 2008]
Season_2009 = df[df['Season Start'] == 2009]
Season_2010 = df[df['Season Start'] == 2010]
Season_2011 = df[df['Season Start'] == 2011]
Season_2012 = df[df['Season Start'] == 2012]
Season_2013 = df[df['Season Start'] == 2013]
Season_2014 = df[df['Season Start'] == 2014]
Season_2015 = df[df['Season Start'] == 2015]
Season_2016 = df[df['Season Start'] == 2016]
Season_2017 = df[df['Season Start'] == 2017]
Season_2018 = df[df['Season Start'] == 2018]
Season_2019 = df[df['Season Start'] == 2019]
Season_2020 = df[df['Season Start'] == 2020]
Season_2021 = df[df['Season Start'] == 2021]
Season_2022 = df[df['Season Start'] == 2022]

# Creating a list of the masked seasons
Seasons_list  = [Season_1998, Season_1999, Season_2000, Season_2001, Season_2002, Season_2003, Season_2005, 
                 Season_2006, Season_2007, Season_2008, Season_2010, Season_2011, Season_2012,
                 Season_2013, Season_2014, Season_2015, Season_2016, Season_2017, Season_2018, 
                 Season_2019, Season_2020, Season_2021, Season_2022]

In [ ]:
# Creating a Dataframe of winners
winners_df = df[df['Stanley Cup'] == 1]

In [ ]:
# Visually plotting Goals vs Wins
plt.figure(figsize = (15, 30))

for i in range(0, len(Seasons_list)):
    
    plt.subplot(6,4,i+1)
    plt.scatter(Seasons_list[i]['P'], Seasons_list[i]['W'], c = Seasons_list[i]['Stanley Cup'], cmap = 'jet')

    plt.xlabel("Goals")
    plt.ylabel("Wins")

plt.tight_layout()

In [ ]:
# Modeling the data using Ordinary Least Squares
independent_variables = df.drop(columns = ['Team','T','OT','RW', 'ROW','S/O Win', 'Stanley Cup', 'Season Start', 'Season End', 'GP','P', 'PP%', 'PK%', 'GF/GP', 'GA/GP', 'GA', 'GF','SA/GP', 'L'] )
X = sm.add_constant(independent_variables)

model = sm.OLS(df['Stanley Cup'], independent_variables)
results = model.fit()
results.summary()

The R-Squared value and the adjusted R-Squared value are very low for different combinations of the dataset. Either we need to gather more data to analyze or we need to change the method that we analyze the data with.

## Sources

https://datatofish.com/read_excel/

https://www.nhl.com/stats/teams?aggregate=0&reportType=season&seasonFrom=19171918&seasonTo=20232024&gameType=2&filter=gamesPlayed,gte,1&sort=faceoffWinPct&page=0&pageSize=50

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

https://www.nhl.com/news/nhl-stanley-cup-champions-winners-complete-list-287705398

https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/

https://seaborn.pydata.org/examples/scatterplot_categorical.html

https://seaborn.pydata.org/examples/scatter_bubbles.html

https://python-charts.com/seaborn/themes/#:~:text=The%20seaborn%20library%20provides%20five,the%20set_style%20or%20set_style%20function.

https://datatofish.com/csv-to-excel-python/